In [ ]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Context

In [ ]:
var log = await Import.FromFile("../Files/900.TransactionalData/NominalCashflows_CH_2020_12.csv").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [ ]:
var log = await Import.FromFile("../Files/900.TransactionalData/Actuals_CH_2020_12.csv").WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [ ]:
var defaultVarsBE = Workspace.Query<IfrsVariable>().ToArray();

# Scenario Cashflows

In [ ]:
var cashflows_MTUP10pct = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,-110,0,0,-110,0,0,-110,0,0,-110,0,0,0,-110,0,0,-110,0,0,-110,0,0,-110,0
DT1.1,NIC,CL,C,,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5
DT1.1,CU,CL,C,,11,11,11,11,11,11,11,11,11,11,11,11,11,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,3.3
DT1.1,RA,CL,C,,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75";

In [ ]:
var cashflows_MTUP10pct_BE = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,-100,0,0,-100,0,0,-100,0,0,-100,0,0,0,-100,0,0,-100,0,0,-100,0,0,-100,0
DT1.1,NIC,CL,C,,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
DT1.1,CU,CL,C,,10,10,10,10,10,10,10,10,10,10,10,10,10,5,5,5,5,5,5,5,5,5,5,3
DT1.1,RA,CL,C,,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5";

In [ ]:
var log = await Import.FromString(cashflows_MTUP10pct).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [ ]:
class IfrsVariableComparer: IEqualityComparer<IfrsVariable>
{
    private bool IgnoreValue;
    private IfrsVariableComparer(bool ignoreValue)
    {
        IgnoreValue = ignoreValue;
    }

    public bool Equals(IfrsVariable x, IfrsVariable y) =>
        x.AccidentYear == y.AccidentYear && x.AmountType == y.AmountType && x.DataNode == y.DataNode && x.AocType == y.AocType && 
        x.Novelty == y.Novelty && x.EstimateType == y.EstimateType && (IgnoreValue ? true : Math.Abs(x.Value - y.Value) < Precision); 

    public int GetHashCode(IfrsVariable v) => IgnoreValue  
                                            ? HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty) 
                                            : HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty, v.Value.ToString("F5"));

    public static IfrsVariableComparer Instance(bool ignoreValue = false)
    { 
        return new IfrsVariableComparer(ignoreValue);
    }
}

In [ ]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new{ReportingNode = "CH", Scenario = "MTUP10pct", Year = 2020, Month = 12});
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new{ReportingNode = "CH", Scenario = "MTUP10pct", Year = 2020, Month = 12});
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
diffs.Where(x => (new[]{"A","AA","OA","DA"}).Contains(x.EstimateType)).ToArray().Length.Should().Be(0);
diffs.Where(x => !(new[]{"A","AA","OA","DA"}).Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

In [ ]:
var a = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance()).Where(x => (new[]{"A","AA","OA","DA"}).Contains(x.EstimateType)).ToArray(); a

In [ ]:
var b = Workspace.Query<IfrsVariable>().Where(x => (new[]{"A","AA","OA","DA"}).Contains(x.EstimateType) && x.DataNode == "DT1.1").ToArray(); b

In [ ]:
var c = defaultVarsBE.Where(x => (new[]{"A","AA","OA","DA"}).Contains(x.EstimateType) && x.DataNode == "DT1.1").ToArray(); c

In [ ]:
// DISCUSS: some Actuals are missing...

# Scenario Actuals

In [ ]:
var actuals_MTUP10pct = @"
@@Main
ReportingNode,Year,Month,Scenario,
CH,2020,12,MTUP10pct,
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,308";

In [ ]:
var actuals_MTUP10pct_BE = @"
@@Main
ReportingNode,Year,Month,Scenario,
CH,2020,12,MTUP10pct,
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,280";

In [ ]:
var log = await Import.FromString(actuals_MTUP10pct).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [ ]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new{ReportingNode = "CH", Scenario = "MTUP10pct", Year = 2020, Month = 12});
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new{ReportingNode = "CH", Scenario = "MTUP10pct", Year = 2020, Month = 12});
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
diffs.Where(x => (new[]{"A","AA","OA","DA"}).Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);

# Robustness: back to Best Estimate

In [ ]:
var log = await Import.FromString(actuals_MTUP10pct_BE).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [ ]:
var log = await Import.FromString(cashflows_MTUP10pct_BE).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [ ]:
Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance()).ToArray().Length.Should().Be(0);